## Setup (run once only)

### 1. Use Conda to create the environment with the following command.

In [1]:
%cd ..

/data2/reach/francis/MMT_experiments/fork/mmt


In [ ]:
!conda env create -f environment.yml

## Pre-trained model and train-args.json (run once only)

Download from https://drive.google.com/drive/folders/15ji-jGE4GICLMpEwlPRXb-Dv_RYqKunH?usp=sharing using gdown

In [2]:
import os

# make the folder exp/midi_YX if it doesn't exist
os.makedirs('exp/midi_YX/', exist_ok=True)

# make the folder exp/midi_Y if it doesn't exist
os.makedirs('exp/midi_Y/', exist_ok=True)

In [3]:
!gdown '15ji-jGE4GICLMpEwlPRXb-Dv_RYqKunH' --folder --output exp/midi_Y/

Retrieving folder list
Retrieving folder 1HjbBGXUE1HurRKgOlY0CpcvQXVvf8Hvr checkpoints
Processing file 1BFG8yj-okko5SX30zunAGbI0L8ZaQRas best_model.pt
Processing file 1b8x6Uq5br87VRMhuUOeAzXc171Szj-kK train-args.json
Retrieving folder list completed
Building directory structure
Building directory structure completed
Access denied with the following error:

 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1BFG8yj-okko5SX30zunAGbI0L8ZaQRas 

Download ended unsuccessfully


In [ ]:
!cp -R exp/midi_Y/* exp/midi_YX/

## Input

The input file should be a MIDI file with 2 tracks - track 1 being Y (melody) and track 2 being X (accompaniment). 

In [4]:
import muspy

input_midi_file_path = '<INPUT_FILE_PATH>' # eg. '../input.mid'

# copy to data/midi_YX/YX/YX.mid
os.makedirs('data/midi_YX/YX', exist_ok=True)
os.system('cp {0} data/midi_YX/YX/YX.mid'.format(input_midi_file_path))

# save track 1 to data/midi_Y/Y/Y.mid
os.makedirs('data/midi_Y/Y', exist_ok=True)
music = muspy.read_midi(input_midi_file_path, backend='pretty_midi') 
music.tracks = [music.tracks[0]]
muspy.write_midi('data/midi_Y/Y/Y.mid', music, backend='pretty_midi')

Prepare list of names

In [5]:
!find data/midi_Y/Y -type f -name *.mid -o -name *.xml | cut -c 15- > data/midi_Y/original-names.txt

In [6]:
!find data/midi_YX/YX -type f -name *.mid -o -name *.xml | cut -c 17- > data/midi_YX/original-names.txt

Convert to JSON

In [7]:
# make the folder data/midi_Y/processed/json if it doesn't exist
os.makedirs('data/midi_Y/processed/json', exist_ok=True)

# make the folder data/midi_YX/processed/json if it doesn't exist
os.makedirs('data/midi_YX/processed/json', exist_ok=True)

In [8]:
!python mmt/convert_lmd_full.py -n data/midi_Y/original-names.txt -i data/midi_Y/Y -o data/midi_Y/processed/json/

INFO     Using arguments:
{'ignore_exceptions': False,
 'in_dir': PosixPath('/data2/reach/francis/MMT_experiments/fork/mmt/data/midi_Y/Y'),
 'jobs': 1,
 'names': PosixPath('/data2/reach/francis/MMT_experiments/fork/mmt/data/midi_Y/original-names.txt'),
 'out_dir': PosixPath('/data2/reach/francis/MMT_experiments/fork/mmt/data/midi_Y/processed/json'),
 'quiet': False,
 'resolution': 12,
 'skip_existing': False}
INFO     Loading names...
INFO     Iterating over names...
100%|██████████████████████████████████████| 1/1 [00:00<00:00, 45.49it/s, Y.mid]
INFO     Converted 1 out of 1 files.
INFO     Saved the converted filenames to: /data2/reach/francis/MMT_experiments/fork/mmt/data/midi_Y/processed/json-names.txt


In [9]:
!python mmt/convert_lmd_full.py -n data/midi_YX/original-names.txt -i data/midi_YX/YX -o data/midi_YX/processed/json/

INFO     Using arguments:
{'ignore_exceptions': False,
 'in_dir': PosixPath('/data2/reach/francis/MMT_experiments/fork/mmt/data/midi_YX/YX'),
 'jobs': 1,
 'names': PosixPath('/data2/reach/francis/MMT_experiments/fork/mmt/data/midi_YX/original-names.txt'),
 'out_dir': PosixPath('/data2/reach/francis/MMT_experiments/fork/mmt/data/midi_YX/processed/json'),
 'quiet': False,
 'resolution': 12,
 'skip_existing': False}
INFO     Loading names...
INFO     Iterating over names...
100%|█████████████████████████████████████| 1/1 [00:00<00:00, 65.67it/s, YX.mid]
INFO     Converted 1 out of 1 files.
INFO     Saved the converted filenames to: /data2/reach/francis/MMT_experiments/fork/mmt/data/midi_YX/processed/json-names.txt


Extract notes from JSON

In [10]:
# make the folder data/midi_YX/processed/notes if it doesn't exist
os.makedirs('data/midi_YX/processed/notes', exist_ok=True)

# make the folder data/midi_Y/processed/notes if it doesn't exist
os.makedirs('data/midi_Y/processed/notes', exist_ok=True)

In [11]:
!python mmt/extract.py -d midi_Y

INFO     Using arguments:
{'dataset': 'midi_Y',
 'ignore_exceptions': False,
 'in_dir': PosixPath('data/midi_Y/processed/json'),
 'jobs': 1,
 'names': PosixPath('data/midi_Y/processed/json-names.txt'),
 'out_dir': PosixPath('data/midi_Y/processed/notes'),
 'quiet': False}
INFO     Saved the encoding to: data/midi_Y/processed/notes/encoding.json
INFO     Loading names...
INFO     Iterating over names...
100%|█████████████████████████████████████| 1/1 [00:00<00:00, 743.01it/s, Y.mid]
INFO     Extracted 1 out of 1 files.
INFO     Saved the extracted filenames to: data/midi_Y/processed/names.txt


In [12]:
!python mmt/extract.py -d midi_YX

INFO     Using arguments:
{'dataset': 'midi_YX',
 'ignore_exceptions': False,
 'in_dir': PosixPath('data/midi_YX/processed/json'),
 'jobs': 1,
 'names': PosixPath('data/midi_YX/processed/json-names.txt'),
 'out_dir': PosixPath('data/midi_YX/processed/notes'),
 'quiet': False}
INFO     Saved the encoding to: data/midi_YX/processed/notes/encoding.json
INFO     Loading names...
INFO     Iterating over names...
100%|████████████████████████████████████| 1/1 [00:00<00:00, 495.55it/s, YX.mid]
INFO     Extracted 1 out of 1 files.
INFO     Saved the extracted filenames to: data/midi_YX/processed/names.txt


Split training/validation/test sets

In [13]:
!python mmt/split.py -d midi_Y -v 0 -t 1

INFO     Using arguments:
{'dataset': 'midi_Y',
 'names': PosixPath('data/midi_Y/processed/names.txt'),
 'out_dir': PosixPath('data/midi_Y/processed'),
 'quiet': False,
 'ratio_test': 1.0,
 'ratio_valid': 0.0,
 'seed': 0}
INFO     Loading names...
INFO     Loaded 1 names.
INFO     Collected 0 files for training.
INFO     Collected 0 files for validation.
INFO     Collected 1 files for test.


In [14]:
!python mmt/split.py -d midi_YX -v 0 -t 1

INFO     Using arguments:
{'dataset': 'midi_YX',
 'names': PosixPath('data/midi_YX/processed/names.txt'),
 'out_dir': PosixPath('data/midi_YX/processed'),
 'quiet': False,
 'ratio_test': 1.0,
 'ratio_valid': 0.0,
 'seed': 0}
INFO     Loading names...
INFO     Loaded 1 names.
INFO     Collected 0 files for training.
INFO     Collected 0 files for validation.
INFO     Collected 1 files for test.


## Generate samples and logits

In [15]:
!python mmt/generate.py -d midi_YX -o exp/midi_YX/ape -g 0 -ns 1

Running command: python mmt/generate.py -d midi_YX -o exp/midi_YX/ape -g 0 -ns 1
Using arguments:
{'dataset': 'midi_YX',
 'filter': 'top_k',
 'filter_threshold': 0.9,
 'gpu': 0,
 'in_dir': PosixPath('data/midi_YX/processed/notes'),
 'jobs': 1,
 'model_steps': None,
 'n_samples': 1,
 'names': PosixPath('data/midi_YX/processed/test-names.txt'),
 'out_dir': PosixPath('/data2/reach/francis/MMT_experiments/fork/mmt/exp/midi_YX/ape'),
 'quiet': False,
 'seq_len': 1024,
 'shuffle': False,
 'temperature': 1.0,
 'use_csv': False}
Saved arguments to /data2/reach/francis/MMT_experiments/fork/mmt/exp/midi_YX/ape/generate-args.json
Loading training arguments from: /data2/reach/francis/MMT_experiments/fork/mmt/exp/midi_YX/ape/train-args.json
Using loaded arguments:
{'abs_pos_emb': True,
 'aug': True,
 'batch_size': 8,
 'dataset': 'sod',
 'dim': 512,
 'dropout': 0.2,
 'early_stopping': True,
 'early_stopping_tolerance': 20,
 'gpu': 7,
 'grad_norm_clip': 1.0,
 'heads': 8,
 'in_dir': 'data/sod/processe

In [16]:
!python mmt/generate.py -d midi_Y -o exp/midi_Y/ape -g 0 -ns 1

Running command: python mmt/generate.py -d midi_Y -o exp/midi_Y/ape -g 0 -ns 1
Using arguments:
{'dataset': 'midi_Y',
 'filter': 'top_k',
 'filter_threshold': 0.9,
 'gpu': 0,
 'in_dir': PosixPath('data/midi_Y/processed/notes'),
 'jobs': 1,
 'model_steps': None,
 'n_samples': 1,
 'names': PosixPath('data/midi_Y/processed/test-names.txt'),
 'out_dir': PosixPath('/data2/reach/francis/MMT_experiments/fork/mmt/exp/midi_Y/ape'),
 'quiet': False,
 'seq_len': 1024,
 'shuffle': False,
 'temperature': 1.0,
 'use_csv': False}
Saved arguments to /data2/reach/francis/MMT_experiments/fork/mmt/exp/midi_Y/ape/generate-args.json
Loading training arguments from: /data2/reach/francis/MMT_experiments/fork/mmt/exp/midi_Y/ape/train-args.json
Using loaded arguments:
{'abs_pos_emb': True,
 'aug': True,
 'batch_size': 8,
 'dataset': 'sod',
 'dim': 512,
 'dropout': 0.2,
 'early_stopping': True,
 'early_stopping_tolerance': 20,
 'gpu': 7,
 'grad_norm_clip': 1.0,
 'heads': 8,
 'in_dir': 'data/sod/processed/notes'

## Compute DINE

In [17]:
import numpy as np

tuple_encoding = {"type" : 0,
                  "beat" : 1, 
                  "position": 2,
                  "pitch" : 3,
                  "duration" : 4,
                  "instrument": 5}

def read_logits(folder):
    """
    The logits folder (eg. /exp/midi_data_x/ape/samples/logits/) will have files like:
    0_16-beat-continuation-beat_logits.npy
    0_16-beat-continuation-duration_logits.npy
    0_16-beat-continuation-instrument_logits.npy
    0_16-beat-continuation-pitch_logits.npy
    0_16-beat-continuation-position_logits.npy
    0_16-beat-continuation-type_logits.npy
    1_16-beat-continuation-beat_logits.npy
    1_16-beat-continuation-duration_logits.npy
    1_16-beat-continuation-instrument_logits.npy
    1_16-beat-continuation-pitch_logits.npy
    1_16-beat-continuation-position_logits.npy
    1_16-beat-continuation-type_logits.npy
    .
    .
    .
    n_16-beat-continuation-beat_logits.npy
    n_16-beat-continuation-duration_logits.npy
    n_16-beat-continuation-instrument_logits.npy
    n_16-beat-continuation-pitch_logits.npy
    n_16-beat-continuation-position_logits.npy
    n_16-beat-continuation-type_logits.npy

    This function will read the logits and return a dictionary of the form:
    {0 : 
        {"type" : np.array(),
        "beat" : np.array(),
        "position" : np.array(),
        "pitch" : np.array(),
        "duration" : np.array(),
        "instrument" : np.array()},
    1 :
        {"type" : np.array(),
        "beat" : np.array(),
        "position" : np.array(),
        "pitch" : np.array(),
        "duration" : np.array(),
        "instrument" : np.array()},
    .
    .
    .
    n : 
        {"type" : np.array(),
        "beat" : np.array(),
        "position" : np.array(),
        "pitch" : np.array(),
        "duration" : np.array(),
        "instrument" : np.array()}
    }
    """
    filenames = os.listdir(folder)
    sample_ids = [int(fname.split('_')[0]) for fname in filenames]
    logits = {id: dict() for id in sample_ids}

    for fname in filenames:
        if fname.endswith(".npy"):
            id = int(fname.split('_')[0])
            l_type = fname.split('-')[-1].split('_')[0]
            logits[id][l_type] = np.load(os.path.join(folder, fname)).squeeze()
    
    return logits

def read_truth(folder):
    """
    The npy folder (eg. /exp/midi_data_x/ape/samples/npy/) will have files like:
    0_truth.npy
    1_truth.npy
    .
    .
    .
    n_truth.npy 

    in addition to the generated files.

    This function will read the truth files and return a dictionary of the form:
    {0 : np.array(),
    1 : np.array(),
    .
    .
    .
    n : np.array()}
    """
    filenames = os.listdir(folder)
    
    truths = dict()
    for fname in filenames:
        if fname.endswith("_truth.npy"):
            id = int(fname.split('_')[0])
            truths[id] = np.load(os.path.join(folder, fname))

    return truths

def calc_9(logits, truth, add_noise='all'):
    """
    Given logits[i] of the form:

    {"type" : np.array(),
    "beat" : np.array(),
    "position" : np.array(),
    "pitch" : np.array(),
    "duration" : np.array(),
    "instrument" : np.array()}

    and truth[i] of the form: np.array(),

    this function will compute the entropy each of the 6 positions in the tuple representation:

    { "type" : entropy,
      "beat" : entropy,
      "position" : entropy,
      "pitch" : entropy,
      "duration" : entropy,
      "instrument" : entropy}
    """ 

    assert add_noise in ['all', 'pitch', 'duration', 'instrument', 'type', 'beat', 'position']

    # select notes from truth after the first 16 beats
    # selected_truth = truth[truth[:, 1] >= 16]

    # select notes from truth after the first 32 elements
    selected_truth = truth[32:]

    # select rows from logits corresponding to the selected notes
    selected_logits = {l_type : logits[l_type][:len(selected_truth)] 
                       for l_type in logits.keys()}

    ######### not converting logits to probabilities #########
    probs = {l_type : selected_logits[l_type] for l_type in selected_logits.keys()}

    # select the probability of the truth note from probs
    selected_probs = {l_type : probs[l_type][np.arange(len(selected_truth)), 
                                                       selected_truth[:, tuple_encoding[l_type]]]
                        for l_type in probs.keys()}

    # compute first_term
    first_term = {l_type : np.mean(selected_probs[l_type]) for l_type in selected_probs.keys()}

    # add noise
    noise = {}
    if add_noise == 'all':
        noise_keys = ['pitch', 'duration', 'beat', 'position']
        r = np.random.RandomState(42)
        for l_type in noise_keys:
            choices = [r.choice(list(set(range(0, probs[l_type].shape[1])).difference(set([selected_truth[i, tuple_encoding[l_type]]])))) 
                       for i in range(len(selected_truth))]
            noise[l_type] = np.array(choices)
    else:
        noise_keys = [add_noise]
        r = np.random.RandomState(42)
        choices = [r.choice(list(set(range(0, probs[add_noise].shape[1])).difference(set([selected_truth[i, tuple_encoding[add_noise]]])))) 
                       for i in range(len(selected_truth))]
        noise[add_noise] = np.array(choices)

    noisy_probs = selected_probs
    for l_type in noise_keys:
        noisy_probs[l_type] = probs[l_type][np.arange(len(selected_truth)), noise[l_type]] 

    # compute second_term
    second_term = {l_type : np.log(np.mean(np.exp(noisy_probs[l_type]))) for l_type in noisy_probs.keys()}

    # compute 9  
    nine = {l_type : first_term[l_type] - second_term[l_type] for l_type in first_term.keys()}

    return nine

In [18]:
# read ground truth
truths = read_truth('exp/midi_Y/ape/samples/npy/')

for key in sorted(truths.keys()):
    print(key, truths[key].shape)

0 (124, 6)


In [19]:
# read logits for Y
logits = read_logits('exp/midi_Y/ape/samples/logits/')

for key in sorted(logits.keys()):
    print(key)
    for l_type in sorted(logits[key].keys()):
        print(l_type, logits[key][l_type].shape)

0
beat (93, 257)
duration (93, 33)
instrument (93, 65)
pitch (93, 129)
position (93, 13)
type (93, 5)


In [20]:
# read logits for YX
logits_YX = read_logits('exp/midi_YX/ape/samples/logits/')

for key in sorted(logits_YX.keys()):
    print(key)
    for l_type in sorted(logits_YX[key].keys()):
        print(l_type, logits_YX[key][l_type].shape)

0
beat (171, 257)
duration (171, 33)
instrument (171, 65)
pitch (171, 129)
position (171, 13)
type (171, 5)


In [21]:
# compute DINE
for id in sorted(logits.keys()):
    print(id, ':')
    nine_a = calc_9(logits[id], truths[id])
    nine_b = calc_9(logits_YX[id], truths[id])
    for key in sorted(['beat', 'position', 'pitch', 'duration']):
        print()
        print(key, ':')
        print('9b - 9a:', nine_b[key] - nine_a[key])
        print('9a:', nine_a[key])
        print('9b:', nine_b[key])
    print('------------------')

0 :

beat :
9b - 9a: -16.91203
9a: 9.603898
9b: -7.308131

duration :
9b - 9a: -3.3433762
9a: 1.435261
9b: -1.9081153

pitch :
9b - 9a: -2.1911764
9a: 3.0902953
9b: 0.8991189

position :
9b - 9a: -6.253174
9a: 3.8853662
9b: -2.3678079
------------------
